<div id="container" style="position:relative;">
<div style="float:left"><h1> Capstone Project Modeling: Funk SVD  </h1></div>
<div style="position:relative; float:right"><img style="height:65px" src ="https://drive.google.com/uc?export=view&id=1EnB0x-fdqMp6I5iMoEBBEuxB_s7AmE2k" />
</div>
</div>

<br>
<br>
<br>


### Camilo Salazar <br> BrainStation <br> November 10, 2023

## Introduction

In this notebook, we delve into the exciting realm of hybrid recommendation systems, a powerful approach that combines the strengths of both collaborative filtering and content-based methods to provide highly personalized book recommendations. By fusing user behavior and content attributes, we aim to create a recommendation model that offers superior accuracy and enhanced user experiences. Join us on this journey as we explore the fusion of data-driven insights and content analysis to bring you a state-of-the-art hybrid book recommender.


In [1]:
# imports usefull libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import random

# import Supriside to run model 
from surprise import Dataset
from surprise.reader import Reader
from surprise.prediction_algorithms.matrix_factorization import SVD as FunkSVD
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise import accuracy

# Filter warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [179]:
#loading all dataframes
book_df = pd.read_csv('data_clean/books.csv',index_col=[0])
tags_df = pd.read_csv('data/tags.csv')
book_tags_df = pd.read_csv('data_clean/book_tags.csv',index_col=[0])
ratings_df = pd.read_csv('data/ratings.csv')

---
## Collaborite Filtering Recomender 

In [3]:
# set reader of the rating
reader = Reader(rating_scale=(1, 5))
my_data = Dataset.load_from_df(ratings_df, reader)
trainset, testset = train_test_split(my_data, test_size=.10, random_state = 42)

In [4]:
final_model = FunkSVD( n_factors = 20,
                 n_epochs = 20,
                 lr_all = 0.0075,
                 biased = False,
                 random_state = 42)

final_model.fit(trainset)

In [5]:
book_latent = final_model.qi
book_latent

array([[ 0.32590215,  0.6499362 , -0.89973914, ..., -0.26051843,
        -0.45952262,  0.6313754 ],
       [-0.22880797,  0.28372632, -1.06872092, ..., -0.14884239,
        -0.26380744,  0.868027  ],
       [ 0.79588949,  0.87041907, -0.52657141, ..., -0.03389259,
         0.5858052 ,  0.47999035],
       ...,
       [ 0.32250818,  0.28507686, -0.84495597, ..., -0.45970672,
        -0.26615837,  0.61314426],
       [ 0.53799812, -0.03097239, -1.02789254, ..., -0.28284891,
        -0.25884952,  0.3056886 ],
       [ 0.22194255,  0.25411235, -1.07073524, ..., -0.60164437,
        -0.23986232,  0.53010669]])

In [6]:
book_simind = pd.DataFrame(list(trainset._raw2inner_id_items.items()
),columns=['book_id', 'Vindex']).set_index('book_id', drop=True)
book_simind

,Vindex
book_id,
2757,0
134,1
1463,2
71,3
3339,4
...,...
7636,9995
9080,9996
9980,9997


In [264]:
def id_bookinfo(b_id):
    '''
    Retrieves book information based on a given book ID.

    Parameters
    ----------
    b_id: int
        The book ID (an integer) for the book to retrieve information about. Should be between 1 and 10,000.

    Returns
    -------
    result: pandas.DataFrame
        A DataFrame containing information about the book, including title, authors, and original publication year.
    '''
    # checks that the book id is and int and valid
    if (not isinstance(b_id, int)) or ((b_id < 1) or (b_id > 10000)):
        raise ValueError("Invalid Book Id Pick an Integer between 1-10000")
    
    result = book_df[book_df['book_id'] == b_id][['title', 'authors', 'original_publication_year','genres']]
    return result

def title_to_id(b_title):
    '''
    Retrieves the book ID based on a given book title.

    Parameters
    ----------
    b_title: str
        The title of the book to find the corresponding book ID for.

    Returns
    -------
    result: int
        The book ID associated with the given title.
    '''
    
    result = book_df[book_df['title'] == b_title]['book_id']
    return result.values[0]

def vin_to_id(vin, book_simind = book_simind):
    '''
    Retrieves the book ID based on a given Vindex (index in a similarity matrix).

    Parameters
    ----------
    vin: int
        The Vindex value representing a book's index in a similarity matrix.

    Returns
    -------
    result: int
        The book ID associated with the given Vindex.
    '''
    result = book_simind[book_simind['Vindex'] == vin].index[0]
    return result

 
def id_to_vin(inb, book_simind = book_simind):
    '''
    Retrieves the Vindex (index in a similarity matrix) based on a given book ID.

    Parameters
    ----------
    inb: int
        The book ID for which the Vindex is needed.

    Returns
    -------
    result: int
    The Vindex associated with the given book ID
    '''     
    
    result = book_simind.loc[inb][0]
    return result

def title_to_vin(b_title, book_simind = book_simind):
    '''
    Retrieves the Vindex (index in a similarity matrix) based on a given book title.

    Parameters
    ----------
    b_title: str
        The title of the book for which the Vindex is needed.

    Returns
    -------
    result: int
        The Vindex associated with the given book title.
    '''
        
    result = id_to_vin(title_to_id(b_title),book_simind = book_simind)
    return result
    
    

In [18]:
from sklearn.metrics.pairwise import cosine_similarity 
coll_similarities = cosine_similarity(book_latent, dense_output=False)

In [86]:
def coll_recomender(b_title, sim_arr, bi):
    '''
    Recommends books similar to a given book title based on book similarities.

    Parameters
    ----------
    b_title: str
        The title of the book for which you want book recommendations.
    book_similarities: np.ndarray
        A 2D numpy array containing book similarities where each row represents a book.
    bi: pd.DataFrame
        A DataFrame containing book indices.

    Returns
    -------
    results: pd.DataFrame
        A DataFrame containing book recommendations and their similarities to the input book.
    '''
    
    # Create a copy of the book indices DataFrame
    botoind = bi.copy()
    # Get the Vindex (index in similarity matrix) of the input book title
    vin = title_to_vin(b_title,bi)
    # Extract similarity data for the input book
    data = sim_arr[vin]
    # Add the Similarities column to the book indices DataFrame
    botoind['Similarities'] = data
    # Sort books by similarity in descending order
    botoind.sort_values('Similarities', ascending=False, inplace=True)
    # Remove the input book from the recommendations
    botoind = botoind.drop(vin_to_id(vin,bi))
    # Get the top 10 book indices
    top10ind = botoind.head(10).index
    results = pd.DataFrame([])
    # Retrieve book information for the top 10 recommended books
    for b_id in top10ind:
        res = id_bookinfo(b_id)
        results = pd.concat([results, res])
    # Add the Similarities column to the recommendations DataFrame
    results['Similarities'] = botoind.head(10)['Similarities'].values
    return results


In [180]:
recomender("Words of Radiance (The Stormlight Archive, #2)",coll_similarities,book_simind)

,title,authors,original_publication_year,Similarities
561,"The Way of Kings (The Stormlight Archive, #1)",Brandon Sanderson,2010.0,0.987111
1340,"Golden Son (Red Rising, #2)",Pierce Brown,2015.0,0.971419
306,"The Wise Man's Fear (The Kingkiller Chronicle,...",Patrick Rothfuss,2011.0,0.969749
140,The Martian,Andy Weir,2012.0,0.968499
9140,"The Way of Kings, Part 1 (The Stormlight Archi...",Brandon Sanderson,2011.0,0.964987
1373,"A Memory of Light (Wheel of Time, #14)","Robert Jordan, Brandon Sanderson",2012.0,0.964794
1807,"Morning Star (Red Rising, #3)",Pierce Brown,2016.0,0.963389
1199,"The Alloy of Law (Mistborn, #4)",Brandon Sanderson,2011.0,0.962975
2888,"Mistborn Trilogy Boxed Set (Mistborn, #1-3)",Brandon Sanderson,2009.0,0.959330
9523,"The Locket (The Locket, #1)",Richard Paul Evans,1998.0,0.959034


In [181]:
recomender("The Martian",coll_similarities,book_simind)

,title,authors,original_publication_year,Similarities
861,"Words of Radiance (The Stormlight Archive, #2)",Brandon Sanderson,2014.0,0.968499
7492,Odd and the Frost Giants,"Neil Gaiman, Mark Buckingham",2008.0,0.968318
4695,"The Defector (Gabriel Allon, #9)",Daniel Silva,2009.0,0.966090
9216,The Cobra Event,Richard Preston,1997.0,0.964894
3197,"The Concrete Blonde (Harry Bosch, #3; Harry Bo...",Michael Connelly,1994.0,0.963949
8707,"Fated (Alex Verus, #1)",Benedict Jacka,2012.0,0.962762
7497,"The Last Full Measure (The Civil War Trilogy, #3)",Jeff Shaara,1991.0,0.962293
6171,"Heat Lightning (Virgil Flowers, #2)",John Sandford,2008.0,0.962248
6719,"Locke & Key, Vol. 4: Keys to the Kingdom","Joe Hill, Gabriel Rodríguez",2011.0,0.962139
9517,"Counterfeit Gods: The Empty Promises of Money,...",Timothy J. Keller,2009.0,0.962038


---
## Content Filtering Recomender 

In [267]:
#Reducing number of tags to those that apeear i over 10 books but less than 66% of them 
tag_list = book_tags_df['tag_id'].value_counts()/book_df.shape[0]*100

tag_list = list(tag_list[(tag_list > 0.1) & (tag_list < 66)].index)

book_tags_df2 = book_tags_df[(book_tags_df['tag_id'].isin(tag_list))]

book_tags_df3 = pd.merge(book_tags_df2, book_df[['book_id','goodreads_book_id']] ,on ='goodreads_book_id',how = 'left')

book_tags_df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768559 entries, 0 to 768558
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype
---  ------             --------------   -----
 0   goodreads_book_id  768559 non-null  int64
 1   tag_id             768559 non-null  int64
 2   count              768559 non-null  int64
 3   book_id            768559 non-null  int64
dtypes: int64(4)
memory usage: 29.3 MB


In [268]:
unq_book = book_tags_df3['goodreads_book_id'].unique()
book_tags_df4 = book_tags_df3.copy()
for i in unq_book:
    max_con = book_tags_df4[book_tags_df4['goodreads_book_id'] == i]['count'].max()
    book_tags_df4.loc[book_tags_df4['goodreads_book_id'] == i,'count'] /= max_con
    
book_tags_df4.tail()

,goodreads_book_id,tag_id,count,book_id
768554,33288638,29299,0.018182,8892
768555,33288638,2101,0.018182,8892
768556,33288638,21303,0.018182,8892
768557,33288638,17271,0.018182,8892
768558,33288638,1126,0.018182,8892


In [269]:
book_tag_max = book_tags_df4.groupby(['book_id','tag_id']).sum().unstack().fillna(0)
book_tag_max.head()

goodreads_book_id                                                  \
tag_id              27    47    71    90    98    115   134   177   190     
book_id                                                                     
1                     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2                     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3                     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4                     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
5                     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

               ... count                                                        
tag_id  192    ... 34003 34011 34031 34051 34148 34153 34155 34157 34206 34242  
book_id        ...                                                              
1         0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2         0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3         0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4         0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
5         0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[5 rows x 10280 columns]

In [270]:
from sklearn.preprocessing import StandardScaler

scale_model =  StandardScaler()

scale_m = scale_model.fit_transform(book_tag_max)

In [271]:
cont1_similarities = cosine_similarity(scale_m, dense_output=False)

In [272]:
tag_ind = book_tag_max.copy()
tag_ind['Vindex'] = range(10000)
tag_ind = tag_ind[['Vindex']]
tag_ind.head()

,Vindex
tag_id,
book_id,
1,0
2,1
3,2
4,3
5,4


In [273]:
tag_ind.tail()

,Vindex
tag_id,
book_id,
9996,9995
9997,9996
9998,9997
9999,9998
10000,9999


In [276]:
coll_recomender("Words of Radiance (The Stormlight Archive, #2)", cont1_similarities, tag_ind)

,title,authors,original_publication_year,genres,Similarities
561,"The Way of Kings (The Stormlight Archive, #1)",Brandon Sanderson,2010.0,"['fantasy', 'fiction']",0.764004
3340,"The Bands of Mourning (Mistborn, #6)",Brandon Sanderson,2016.0,"['fantasy', 'fiction']",0.518539
1199,"The Alloy of Law (Mistborn, #4)",Brandon Sanderson,2011.0,"['fantasy', 'fiction', 'science-fiction']",0.515263
2159,"The Blinding Knife (Lightbringer, #2)",Brent Weeks,2012.0,"['fantasy', 'fiction']",0.473415
2791,"Shadows of Self (Mistborn, #5)",Brandon Sanderson,2015.0,"['fantasy', 'fiction', 'mystery']",0.464496
7992,"Secret History (Mistborn, #3.5)",Brandon Sanderson,2016.0,"['fantasy', 'fiction']",0.374148
2977,The Emperor's Soul,Brandon Sanderson,2012.0,"['fantasy', 'fiction']",0.342964
3724,"Rise of Empire (The Riyria Revelations, #3-4)",Michael J. Sullivan,2010.0,"['fantasy', 'fiction']",0.338344
3797,"Heir of Novron (The Riyria Revelations, #5-6)",Michael J. Sullivan,2012.0,"['fantasy', 'fiction']",0.335325
3533,"The Broken Eye (Lightbringer, #3)",Brent Weeks,2014.0,"['fantasy', 'fiction']",0.315290


In [282]:
coll_recomender('The Martian', cont1_similarities, tag_ind)

,title,authors,original_publication_year,genres,Similarities
2840,Packing for Mars: The Curious Science of Life ...,Mary Roach,2010.0,"['nonfiction', 'science', 'history']",0.272874
7603,"The Quantum Thief (Jean le Flambeur, #1)",Hannu Rajaniemi,2010.0,"['science-fiction', 'fiction', 'fantasy', 'mys...",0.261178
1374,Me and Earl and the Dying Girl,Jesse Andrews,2012.0,"['young-adult', 'contemporary', 'fiction']",0.176037
457,Station Eleven,Emily St. John Mandel,2014.0,"['fiction', 'science-fiction', 'fantasy']",0.172275
339,"I Am Number Four (Lorien Legacies, #1)",Pittacus Lore,2010.0,"['young-adult', 'fantasy', 'science-fiction', ...",0.142495
3726,"The Rook (The Checquy Files, #1)",Daniel O'Malley,2012.0,"['fantasy', 'mystery', 'fiction', 'paranormal'...",0.140132
829,The DUFF: Designated Ugly Fat Friend,Kody Keplinger,2010.0,"['young-adult', 'contemporary', 'romance', 'fi...",0.140095
1735,Dark Matter,Blake Crouch,2016.0,"['science-fiction', 'fiction', 'thriller', 'my...",0.138043
7515,"Mars, Volume 01",Fuyumi Soryo,1996.0,"['manga', 'romance', 'graphic-novels', 'comics...",0.130165
857,"Red Rising (Red Rising, #1)",Pierce Brown,2014.0,"['science-fiction', 'fantasy', 'young-adult', ...",0.125981


In [196]:
book_df['genres']


0       ['young-adult', 'fiction', 'fantasy', 'science...
1       ['fantasy', 'fiction', 'young-adult', 'classics']
2       ['young-adult', 'fantasy', 'romance', 'fiction...
3       ['classics', 'fiction', 'historical-fiction', ...
4       ['classics', 'fiction', 'historical-fiction', ...
                              ...                        
9995      ['fantasy', 'romance', 'paranormal', 'fiction']
9996               ['biography', 'history', 'nonfiction']
9997                    ['historical-fiction', 'fiction']
9998                         ['nonfiction', 'psychology']
9999      ['history', 'nonfiction', 'historical-fiction']
Name: genres, Length: 10000, dtype: object

In [254]:
from sklearn.feature_extraction.text import CountVectorizer

text_model = CountVectorizer(tokenizer=custom_tokenizer)

text_pop = text_model.fit_transform(book_df['genres'])

In [255]:
cont_max = pd.DataFrame(columns = text_model.get_feature_names_out(), data = text_pop.toarray())

In [256]:
cont_max

,art,biography,books,business,chick-lit,christian,classics,comics,contemporary,cookbooks,...,romance,science,science-fiction,self-help,spirituality,sports,suspense,thriller,travel,young-adult
0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9996,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [257]:
cont2_similarities = cosine_similarity(cont_max, dense_output=False)

In [251]:
from  nltk.stem import PorterStemmer
import re

def custom_tokenizer(text):
    text = re.sub("[,'\]\[]", ' ', text)  # Remove non-alphabetic characters
    text = text.lower()  # Convert to lowercase
    # split sentence into words
    listofwords = text.split()
    return listofwords

s = book_df['genres'][0]



In [253]:
turn = custom_tokenizer(s)
turn

['young-adult', 'fiction', 'fantasy', 'science-fiction', 'romance']

--- 
## Hybrid Recomender

In [366]:
def hybrid_recomender(b_title, notauthor = True):
    # get book_id
    book_id = title_to_id(b_title)
    # get both indices 
    vin_coll = id_to_vin(book_id, book_simind)
    vin_cont = id_to_vin(book_id,tag_ind) 
    # get the similarity data
    data_col = coll_similarities[vin_coll]
    data_cont = cont1_similarities[vin_cont]
    b1 = book_simind.copy()
    b2 = tag_ind.copy()
    # get the data to match index and combine to a single dataframe
    b1['SIM1'] =  data_col
    b2['SIM2'] = data_cont
    merge_df = pd.merge(b2, b1, on = 'book_id', how = 'left')
    
    # get the total score
    merge_df['score'] =  merge_df['SIM1'] +  0.25 *merge_df['SIM2']
    
    if notauthor:
        auth = book_df[book_df['book_id'] == book_id]['authors']
        sm_auth_id = list(book_df[book_df['authors'] == auth.values[0]]['book_id'].values)
        merge_df = merge_df.drop(sm_auth_id)
    
    else: merge_df = merge_df.drop(book_id)
    # remove the book from is calculation
    merge_df = merge_df.sort_values('score',ascending=False,)
    top10 =  merge_df.head(10).index
    results = pd.DataFrame([])
    # Retrieve book information for the top 10 recommended books
    for b_id in top10:
        res = id_bookinfo(b_id)
        results = pd.concat([results, res])
    # Add the Similarities column to the recommendations DataFrame
    results['score'] = merge_df.head(10)['score'].values
    return results



In [367]:
hybrid_recomender("Words of Radiance (The Stormlight Archive, #2)",True)

,title,authors,original_publication_year,genres,score
2159,"The Blinding Knife (Lightbringer, #2)",Brent Weeks,2012.0,"['fantasy', 'fiction']",1.061907
306,"The Wise Man's Fear (The Kingkiller Chronicle,...",Patrick Rothfuss,2011.0,"['fantasy', 'fiction']",1.030759
3797,"Heir of Novron (The Riyria Revelations, #5-6)",Michael J. Sullivan,2012.0,"['fantasy', 'fiction']",1.027697
3533,"The Broken Eye (Lightbringer, #3)",Brent Weeks,2014.0,"['fantasy', 'fiction']",1.022105
1340,"Golden Son (Red Rising, #2)",Pierce Brown,2015.0,"['science-fiction', 'fantasy', 'fiction', 'you...",1.005468
6308,"The Crimson Campaign (Powder Mage, #2)",Brian McClellan,2014.0,"['fantasy', 'fiction']",0.997823
3724,"Rise of Empire (The Riyria Revelations, #3-4)",Michael J. Sullivan,2010.0,"['fantasy', 'fiction']",0.993732
1807,"Morning Star (Red Rising, #3)",Pierce Brown,2016.0,"['science-fiction', 'fantasy', 'fiction', 'you...",0.989520
2605,"The Daylight War (Demon Cycle, #3)",Peter V. Brett,2013.0,"['fantasy', 'fiction']",0.988483
7196,The Providence of Fire (Chronicle of the Unhew...,Brian Staveley,2015.0,"['fantasy', 'fiction']",0.984989


In [364]:
hybrid_recomender("Words of Radiance (The Stormlight Archive, #2)",True)

,title,authors,original_publication_year,genres,score
2159,"The Blinding Knife (Lightbringer, #2)",Brent Weeks,2012.0,"['fantasy', 'fiction']",1.061907
306,"The Wise Man's Fear (The Kingkiller Chronicle,...",Patrick Rothfuss,2011.0,"['fantasy', 'fiction']",1.030759
3797,"Heir of Novron (The Riyria Revelations, #5-6)",Michael J. Sullivan,2012.0,"['fantasy', 'fiction']",1.027697
3533,"The Broken Eye (Lightbringer, #3)",Brent Weeks,2014.0,"['fantasy', 'fiction']",1.022105
1340,"Golden Son (Red Rising, #2)",Pierce Brown,2015.0,"['science-fiction', 'fantasy', 'fiction', 'you...",1.005468
6308,"The Crimson Campaign (Powder Mage, #2)",Brian McClellan,2014.0,"['fantasy', 'fiction']",0.997823
3724,"Rise of Empire (The Riyria Revelations, #3-4)",Michael J. Sullivan,2010.0,"['fantasy', 'fiction']",0.993732
1807,"Morning Star (Red Rising, #3)",Pierce Brown,2016.0,"['science-fiction', 'fantasy', 'fiction', 'you...",0.989520
2605,"The Daylight War (Demon Cycle, #3)",Peter V. Brett,2013.0,"['fantasy', 'fiction']",0.988483
7196,The Providence of Fire (Chronicle of the Unhew...,Brian Staveley,2015.0,"['fantasy', 'fiction']",0.984989


In [341]:
auth = book_df[book_df['book_id'] == 562]['authors']

In [351]:
list(book_df[book_df['authors'] == auth.values[0]]['book_id'].values)

[389,
 562,
 565,
 603,
 862,
 970,
 1049,
 1200,
 1665,
 2118,
 2792,
 2889,
 2978,
 3147,
 3249,
 3341,
 4488,
 4745,
 7993,
 8373,
 9141]

In [346]:
auth.values[0]

'Brandon Sanderson'